# Module 1-3 Worksheet: Persistent Homology Pipelines

<font color = 'purple'>
    
*Once we are done talking through the slides from [the main module 1-3 notebook](1-3-PersistencePipelines.ipynb), we will take some time to test out your understanding of the definitions by working through the following questions. We encourage talking to your neighbor, and drawing pictures for understanding! Please ask if you need help!*
    
</font>


In [ ]:
# Run this code once to make the answer widgets work

import ipywidgets as widgets
from IPython.display import display, Image
button = widgets.Button(description="Reveal Answer")

def on_button_clicked(b):
    global filename
    display(Image(filename=filename, width='auto', height='auto'))

In [ ]:
# Standard imports

import numpy as np
import matplotlib.pyplot as plt

from teaspoon.MakeData.PointCloud import Torus, Annulus, Circle
from teaspoon.TDA.Persistence import BettiCurve
from teaspoon.TDA.Draw import drawDgm
from ripser import ripser

The goal of this worksheet is to give you some starter code to build examples, and to build a connection between types of input filtrations and the resulting persistence diagrams. We have two sections:
- Point cloud input data, where we are building a Rips complex filtration; and 
- Matrix/Image data. 

# 1. Input data: Point cloud 

Our first examples come from studying how persistence measures points clouds, particularly ones with a circular structure. 

Consider the following point cloud, which samples from an annulus with inner radius $r$ and outer radius $R$ as in the figure below. 

![](figures/AnnulusWithLabels.png)

In [ ]:
# An annulus and its persistence diagram
A = Annulus(N = 300, r = 2, R = 3)
Dgm = ripser(A,2)['dgms'][1]

# Create subplots
f, (ax1, ax2) = plt.subplots(1, 2, figsize= (12,6))

# Point Cloud
ax1.scatter(*A.T)
ax1.set_title('Point Cloud')
ax1.axis('equal')

# Persistence Dgm
ax2.scatter(Dgm[:, 0], Dgm[:, 1])
ax2.plot([0, 4], [0, 4], '--k')
ax2.axis('equal')
ax2.set_title('Persistence Diagram')

<font color = 'purple'><b>Question:</b></font>
Change the value of $r$ to some other value in $[0,R] = [0,3]$, resimulate the data and check the resulting persistence diagram. What changed? What didn't change? 

*Space for your notes/answer*

*The correct answer is hidden here. Highlight or double-click to open the cell to see it when you're ready. No peeking!*

<font color='white'>
    
The $r$ value changes the size of the inner radius. In this example, the point in the persistence diagram far from the diagonal is directly connected to the size of the inner radius, so changing the $r$ value moves the point far from the diagonal around. Small $r$ means it will nearly disappear into the diagonal while closer to $R=3$ will make it more prominent.

</font>

<font color = 'purple'><b>Question:</b></font>
Reset $r=2$ and test what happens when you change $R$ to be values in $[2,5]$. What changed? What didn't change? 

*Space for your notes/answer*

*The correct answer is hidden here. Highlight or double-click to open the cell to see it when you're ready. No peeking!*

<font color='white'>
  
The value of $R$ spreads out the noise, but doesn't directly affect the high persistence point. So you should see some differences in the noise near the diagonal, but the high persistence point will stay generally stable. 

</font>

<font color = 'purple'><b>Question:</b></font>
Try playing with the $r$ and $R$ parameters in the following example to see what changes.

In [ ]:
# Two annuluses
A1 = Annulus(N = 200, r = 3, R = 4)
A2 = Annulus(N = 200, r = 1, R = 2.5)
A2.T[0, :] = A2.T[0, :] + 5
A = np.concatenate((A1, A2), axis=0)

Dgm = ripser(A,2)['dgms'][1]
# Create subplots
f, (ax1, ax2) = plt.subplots(1, 2, figsize= (12,6))

# Point Cloud
ax1.scatter(*A.T)
ax1.set_title('Point Cloud')
ax1.axis('equal')

# Persistence Dgm
ax2.scatter(Dgm[:, 0], Dgm[:, 1])
ax2.plot([0, 6], [0, 6], '--k')
ax2.axis('equal')
ax2.set_title('Persistence Diagram')

# 2. Input data: Matrix

In this section, we assume that you are given a black and white image (AKA a matrix) as input, and intend to do sublevelset persistence. For the purposes of this worksheet, we will assume we are represenging our sublevelsets at a fixed function value by the top dimensional cells (squares). 

## Sample matrix

Consider the following matrix. Draw the sublevel sets at levels 1, 2, and 3. Compare with answers hidden below

<img src = 'figures/Cubical/Nice_Example_Matrix.png' width = 20% height = 'auto'>


In [ ]:
# Run this cell everytime

display(button)
filename = 'figures/Cubical/Nice_Example.png'
button.on_click(on_button_clicked)

### Code version 

Of course, we won't expect you to be computing persistence on your input data by yourself, so the following blocks will provide code for computing persistence on this filtration for you. 

In [ ]:
# Import Libraries. 
# In this case, we will use giotto-tda for cubical computations

from gtda.homology import CubicalPersistence
from gtda.diagrams import Filtering

In [ ]:
# Here is the code that generates our simple example matrix above.
Density = np.array([[1, 1, 1, 1, 1], 
                    [1, 2, 2, 2, 1], 
                    [1, 2, 3, 2, 1],
                    [1, 2, 2, 2, 1],
                    [1, 1, 1, 1, 1]])
print(Density)

im = plt.matshow(Density)
plt.colorbar()

Now with our simple input example, we can compute the sublevelset persistence using the following code.

In [ ]:
# Find Cubical Persistence

Density = np.array([Density])
cubical_persistence = CubicalPersistence(n_jobs=-1, reduced_homology=True)
Persistence = cubical_persistence.fit_transform(Density)

filtered = Filtering(epsilon=10 ** -2)
Filtered_Persistence = filtered.fit_transform(Persistence)[0]

In [ ]:
Rows = Filtered_Persistence[:,2]==1
Dgm = Filtered_Persistence[Rows,:2]
drawDgm(Dgm)

To see the resulting persistence diagram in case of issues, click the button below. 

In [ ]:
# Run this cell everytime

display(button)
filename = 'figures/Cubical/Nice_Example_PD.png'
button.on_click(on_button_clicked)

Based on your persistence diagram above, test your understanding of the persistence diagram using the following questions.

<font color = 'purple'><b>Question:</b></font>
There is a single point in the persistence diagram. When is the associated class birth time and death time? 

_Your answer_

*The correct answer is hidden here. Highlight or double-click to open the cell to see it when you're ready. No peeking!*

<font color='white'>
    The class is born at 1 and dies at 3. 
</font>

<font color = 'purple'><b>Question:</b></font>
What is the persistence (also called lifetime) of the component?

_Your answer_

*The correct answer is hidden here. Highlight or double-click to open the cell to see it when you're ready. No peeking!*

<font color='white'>
    The lifetime is death time minus birth time, so in this case $3-1=2$.
    
</font>

<font color = 'purple'><b>Question:</b></font>
Can you change the value of one pixel in this matrix in order to increase the lifetime of this class? Which one?

_Your answer_

*The correct answer is hidden here. Highlight or double-click to open the cell to see it when you're ready. No peeking!*

<font color='white'>
    If you increase the value of the center pixel, the death time will increase to whatever value you choose so you can make your lifetime as long as you want. 
</font>

## A more complex example: Sample Density

Below, we have included a more complicated (and more interesting!) function to try out cubical persistence. 

In [ ]:
# Generate grid and function

X1, X2 = np.mgrid[-6:6:250j, -6:6:250j]
Density = np.exp(-0.5 * (X1 ** 2 + X2 ** 2)) + np.exp(-0.5 * ((X1-3) ** 2 + X2 ** 2))
fig = plt.figure(figsize=(6, 6))
ax = plt.axes(projection='3d')
surf = ax.plot_surface(X1, X2, Density, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
plt.show()

The image is included below in case of plotting issues above. 

In [ ]:
# Run this cell everytime

display(button)
filename = 'figures/Cubical/Gaussian.png'
button.on_click(on_button_clicked)

<font color = 'purple'><b>Question:</b></font>
Before attempting to use the code to compute persistence, how many points do you expect to see in the 1-dimensional persistence diagram?

_Your answer_

*The correct answer is hidden here. Highlight or double-click to open the cell to see it when you're ready. No peeking!*

<font color='white'>
   You should expect to see two points, but you're about to go find that out in the next code example anyway. 
</font>

## Use the code below to compute the persistence diagram

In [ ]:
# Find Cubical Persistence

Density = np.array([Density])
cubical_persistence = CubicalPersistence(n_jobs=-1, reduced_homology=True)
Persistence = cubical_persistence.fit_transform(Density)

filtered = Filtering(epsilon=10 ** -2)
Filtered_Persistence = filtered.fit_transform(Persistence)[0]
Rows = Filtered_Persistence[:,2]==1
Dgm = Filtered_Persistence[Rows,:2]
drawDgm(Dgm)

See below in case of plotting or computation issues.

In [ ]:
# Run this cell everytime

display(button)
filename = 'figures/Cubical/Gaussian_PD.png'
button.on_click(on_button_clicked)

<font color = 'purple'><b>Question:</b></font>
Connect the points in the diagram to the features in the original input function. What structural features of the function do the birth and death times correspond to? 

_Your answer_

*The correct answer is hidden here. Highlight or double-click to open the cell to see it when you're ready. No peeking!*

<font color='white'>
    Death is at 1 for both of them
</font>

# Challenge Problem

Here is a more complicated example to try out. Can you connect the points in the persistence diagram to the holes in the filtration induced by the matrix?

In [ ]:
Density = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                   [1, 3, 3, 3, 3, 3, 6, 6, 6, 1],
                   [1, 3, 5, 5, 4, 4, 6, 4, 6, 1],
                   [1, 3, 5, 5, 4, 4, 6, 4, 6, 1],
                   [1, 3, 3, 3, 3, 3, 6, 6, 6, 1],
                   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                   [1, 3, 3, 1, 1, 1, 2, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
print(Density)

plt.imshow(Density)
plt.colorbar()

In [ ]:
# Find Cubical Persistence

Density = np.array([Density])
cubical_persistence = CubicalPersistence(n_jobs=-1, reduced_homology=True)
Persistence = cubical_persistence.fit_transform(Density)

filtered = Filtering(epsilon=10 ** -2)
Filtered_Persistence = filtered.fit_transform(Persistence)[0]
Rows = Filtered_Persistence[:,2]==1
Dgm = Filtered_Persistence[Rows,:2]
drawDgm(Dgm)

In case the code doesn't run for you, click the button below to see the persistence diagram.

In [ ]:
# Run the cell first

display(button)
filename = 'figures/Cubical/Cubical_Example_PD.png'
button.on_click(on_button_clicked)

## Side note: In these modules, we have only found sublevel persistence

### That is: Everything below the threshold is included, everything above is excluded

Here's an animation which computes superlevel persistence, we'll look at these more in Module 2-4 while discussing stochastic bifurcations

![](figures/Stochastic/Cubical.gif)